### 0. Initialization

#### 0.1. Importing all necessary libraries

In [10]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.cluster import KMeans
import scipy as sp
import pickle
import matplotlib.pyplot as plt
from matplotlib import cm
np.random.seed(0)

#### 0.2. Specifying the inputs

In [11]:
agn_num = 22
n_frames = 1200

#### 0.3. Load the high resolution envelope lattice and preferences

In [12]:
lattice_path = os.path.relpath('../Data/dynamic output/voxelized_envelope_cut.csv')
avail_lattice = tg.lattice_from_csv(lattice_path)
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice), avail_lattice)

agn_prefs = pd.read_csv("../Data/raw data/Programme_pref.csv")
agn_prefs

,space_name,space_id,noise_field,dist_entrance,dist_fac,sun_access,skyview
0,Student Housing 1 p,0,0.4,0.55,0.87,0.8,0.6
1,Student Housing 4 p,1,0.4,0.55,0.87,0.8,0.6
2,Assisted Living,2,0.8,0.40,0.93,0.8,0.8
3,Starter Housing,3,0.6,0.60,0.93,0.8,0.6
4,Underground Parking,4,0.8,0.07,0.20,0.0,0.0
5,Bicycle parking,5,0.0,0.85,0.07,0.2,0.0
6,Vegetation/park,6,0.5,0.35,0.53,1.0,0.0
7,Workshops,7,0.3,0.65,0.70,0.4,0.4
8,Fab-Labs,8,0.2,0.50,0.60,0.5,0.4
9,Co-working Space,9,0.5,0.60,0.67,0.4,0.4


#### 0.4. Load the animation frames/lattices

In [13]:
frames = []
for i in range(n_frames):
    csv_path = os.path.relpath('../Data/dynamic output/abm_animation/abm_f_'+ f'{i:03}' + '.csv')
    frames.append(tg.lattice_from_csv(csv_path))

### 1. Visualization

#### 1.1. Visusalize a specific agent 

In [14]:
agn = 1
p = pv.Plotter(notebook=True)

base_lattice = frames[0]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
#p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the availability lattice
init_avail_lattice.fast_vis(p)

# adding axes
p.add_axes()
p.show_bounds(grid="back", location="back", color="#aaaaaa")

def create_mesh(value):
    f = int(value)
    lattice = frames[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([agn-0.1, agn+0.9])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False)
    return

p.add_slider_widget(create_mesh, [0, n_frames], title='Time', value=0, event_type="always", style="classic")
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(275.910778145043, 194.91077814504303, 257.91077778741516),
 (36.0, -45.0, 17.99999964237213),
 (0.0, 0.0, 1.0)]

#### 1.2. Visualize all agents

In [22]:
p = pv.Plotter(notebook=True)

base_lattice = frames[0]

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

# adding the boundingbox wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")

# adding the availability lattice
#init_avail_lattice.fast_vis(p)

# adding axes
#p.add_axes()
#p.show_bounds(grid="back", location="back", color="#aaaaaa")

# making space list with index for the sargs
space_list = agn_prefs.get('space_name')
# formatting for the sarg annotation
space_list = space_list.to_dict()

sargs = dict(
    shadow = True,
    n_labels = 0,
    italic = False,
    fmt=" %.0f",
    font_family="arial",
    height = 0.6,
    vertical = True,
    position_x = 1,
    position_y = 0.75)
    
def create_mesh(value):
    f = int(value)
    lattice = frames[f]

    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F").astype(int)  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([-0.1, agn_num - 0.9])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=True, annotations = space_list, scalar_bar_args=sargs, cmap= "tab20")
    return

p.add_slider_widget(create_mesh, [0, n_frames], title='Time', value=0, event_type="always", style="classic")

#for i in range(120):
   # create_mesh(i * 10)
    #p.screenshot("growth_" + str(i * 10), transparent_background=True)
  
p.show(use_ipyvtk=True)


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(275.910778145043, 194.91077814504303, 257.91077778741516),
 (36.0, -45.0, 17.99999964237213),
 (0.0, 0.0, 1.0)]